<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/fix-training/NN_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [1]:
import os
def is_running_on_colab():
    return "COLAB_GPU" in os.environ

LOCAL = not is_running_on_colab()

In [2]:
if not LOCAL:
    !pip install petastorm -qq
    !pip install pyspark -qq
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

In [3]:
#@title Imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT

from tqdm.notebook import tqdm
import time
import gc

if not LOCAL:
    from google.colab import drive

from typing import Tuple
from functools import reduce
import pickle
import torch
from petastorm import make_batch_reader
from petastorm.pytorch import DataLoader

In [4]:
#@title Set up variables
if not LOCAL:
    JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
    GDRIVE_DIR = "/content/drive"
    GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
    GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
    AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
    LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
    MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
    SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
else:
    GDRIVE_DATA_DIR = os.path.abspath("./data")
    GDRIVE_HOME_DIR = os.path.abspath("./data")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
    JAVA_HOME = "/opt/homebrew/opt/openjdk"
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [5]:
if not LOCAL:
    drive.mount(GDRIVE_DIR, force_remount=True)

In [6]:
#@title Create the session
config = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=config)

23/06/21 03:38:46 WARN Utils: Your hostname, MacBook-Air-di-Domiziano.local resolves to a loopback address: 127.0.0.1; using 192.168.1.175 instead (on interface en0)
23/06/21 03:38:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 03:38:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark = SparkSession.builder.getOrCreate()

# Data acquisition

In [8]:
playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF_PATH = os.path.join(SAVED_DFS_PATH, f"nn_songs_info_df-{NUM_PLAYLISTS}.json")
SONGS_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"nn_songs_vector_length-{NUM_PLAYLISTS}.txt")

ARTIST_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"nn_artist_vector_length-{NUM_PLAYLISTS}.txt")
ARTISTS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-train-{NUM_PLAYLISTS}.json")

In [11]:
# The DF used for train (80% of the original) (playlist are different)
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
# The DF used for testing (20% of the original) (playlist are different)
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

# The DF used for train in the NN model (can be filtered or not)
NN_TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"nn_train_df-{NUM_PLAYLISTS}.json")
# The DF used for testing in the NN model (can be filtered or not)
NN_TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"nn_test_df-{NUM_PLAYLISTS}.json")
# The partition in train test of the NN test set. (Same playlists, different songs)
NN_TEST_DF_TRAIN_PATH = os.path.join(SAVED_DFS_PATH, f"nn_test_df-train-{NUM_PLAYLISTS}.json")
NN_TEST_DF_TEST_PATH = os.path.join(SAVED_DFS_PATH, f"nn_test_df-test-{NUM_PLAYLISTS}.json")

NN_EVAL_PATH = os.path.join(SAVED_DFS_PATH, f"nn_eval_df-{NUM_PLAYLISTS}.json")
NN_EVAL_TRAIN_PATH = os.path.join(SAVED_DFS_PATH, f"nn_eval_df-train-{NUM_PLAYLISTS}.json")
NN_EVAL_TEST_PATH = os.path.join(SAVED_DFS_PATH, f"nn_eval_df-test-{NUM_PLAYLISTS}.json")
# New one:
ARTISTS_EMBEDDINGS_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-train-{NUM_PLAYLISTS}.json")
ARTISTS_EMBEDDINGS_TEST_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-test-train-{NUM_PLAYLISTS}.json")
ARTISTS_EMBEDDINGS_TEST_TEST = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-test-test{NUM_PLAYLISTS}.json")

ARTISTS_EMBEDDINGS_EVAL = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-eval-{NUM_PLAYLISTS}.json")
ARTISTS_EMBEDDINGS_EVAL_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-eval-train-{NUM_PLAYLISTS}.json")
ARTISTS_EMBEDDINGS_EVAL_TEST = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-eval-test{NUM_PLAYLISTS}.json")

# The length of the artist vector length (Artist vectors are only used in the NN model)
ARTIST_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"nn_artist_vector_length-{NUM_PLAYLISTS}.txt")

SONGS_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-{NUM_PLAYLISTS}.txt")
# This may be filtered or not
FILTERED_SONGS_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"nn_songs_vector_length-{NUM_PLAYLISTS}.txt")

SONGS_EMBEDDINGS_TRAIN = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_EMBEDDINGS_TEST = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-test-{NUM_PLAYLISTS}.json")

NN_SONGS_EMBEDDINGS_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-train-{NUM_PLAYLISTS}.json")
NN_SONGS_EMBEDDINGS_TEST_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-test-train-{NUM_PLAYLISTS}.json")
NN_SONGS_EMBEDDINGS_TEST_TEST = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-test-test-{NUM_PLAYLISTS}.json")

NN_SONGS_EMBEDDINGS_EVAL = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-eval-{NUM_PLAYLISTS}.json") #TODO: The logic to produce this still has to be coded.
NN_SONGS_EMBEDDINGS_EVAL_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-eval-train-{NUM_PLAYLISTS}.json")
NN_SONGS_EMBEDDINGS_EVAL_TEST = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-eval-test-{NUM_PLAYLISTS}.json")

SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-{NUM_PLAYLISTS}.json")
FILTERED_SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"nn_songs_info_df-{NUM_PLAYLISTS}.json")

In [12]:
songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
artists_embeddings = spark.read.schema(playlist_schema_mapped).json(ARTISTS_EMBEDDINGS_PATH)
song_mapping = spark.read.json(SONGS_INFO_DF_PATH)

songs_embeddings_eval_train = spark.read.schema(playlist_schema_mapped).json(NN_SONGS_EMBEDDINGS_EVAL_TRAIN)
songs_embeddings_eval_test = spark.read.schema(playlist_schema_mapped).json(NN_SONGS_EMBEDDINGS_EVAL_TEST)

artists_embeddings_eval_train = spark.read.schema(playlist_schema_mapped).json(ARTISTS_EMBEDDINGS_EVAL_TRAIN)
artists_embeddings_eval_test = spark.read.schema(playlist_schema_mapped).json(ARTISTS_EMBEDDINGS_EVAL_TEST)

with open(ARTIST_VECTOR_LENGTH_PATH, "r") as f:
  content = f.read()
  ARTIST_VECTOR_LENGTH = int(content)
with open(SONGS_VECTOR_LENGTH_PATH, "r") as f:
  content = f.read()
  SONGS_VECTOR_LENGTH = int(content)

In [13]:
songs_embeddings.show(), artists_embeddings.show(), ARTIST_VECTOR_LENGTH, SONGS_VECTOR_LENGTH

+------------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|              name|collaborative|  pid|modified_at|num_tracks|num_albums|num_followers|              tracks|num_edits|duration_ms|num_artists|
+------------------+-------------+-----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|               Mix|        false|24770| 1376352000|        67|        43|            1|(133477,[84,3091,...|       32|   16179675|         34|
|         C H I L L|        false|84782| 1508889600|       105|        79|            2|(133477,[256,1951...|       74|   23647221|         59|
|    COACHELLA 2013|        false| 3188| 1366675200|       207|        87|            1|(133477,[142,146,...|       16|   49566298|         41|
|              Heat|        false| 3879| 1492992000|        27|        20|            2|(133477,[1171,372...|       11|    5580836|     

(None, None, 52469, 681805)

# Convert PySpark DataFrame into PyTorch DataLoader

In [14]:
def convert_sparse_to_indices(df: DataFrame, column_name: str) -> DataFrame:
  """
  Given a dataframe fo columns "pos":int and "tracks":SparseVector, it returns a new dataframe where
  the SparseVector are replaced with a list of the indices where the values are.
  (The value information is lost, but we don't care since they are binary values so they will be all ones)
  """

  @F.udf(returnType=ArrayType(IntegerType()))
  def transform_array(item: SparseVector):
    """
    Given a SparseVector (binary) it returns the tuple that represent it, of the type (size, indices)
    """
    indices_list = item.indices.tolist()
    padding_width = max_songs - len(indices_list)
    return indices_list + [-1] * padding_width

  max_songs = songs_embeddings.select(F.max("num_tracks")).first()[0]
  print(f"Max number of songs: {max_songs}")
  df = df.withColumn(f"{column_name}_indices", transform_array(F.col(column_name))).drop(column_name)
  return df


In [15]:
def padded_tensors_to_sparse_matrix(padded_tensor: torch.Tensor, shape: tuple) -> torch.Tensor:
  batch_size, max_songs = padded_tensor.size(0), padded_tensor.size(1)
  rows = []
  for row_idx in range(batch_size):
    row = padded_tensor[row_idx]
    indices = row[row != -1]
    sparse_tensor = torch.sparse_coo_tensor(indices.unsqueeze(0), torch.ones(indices.shape), shape)
    rows.append(sparse_tensor)
  return torch.stack(rows)

def padded_tensors_to_dense_matrix(padded_tensor: torch.Tensor, shape: tuple) -> torch.Tensor:
  batch_size, max_songs = padded_tensor.size(0), padded_tensor.size(1)
  rows = []
  for row_idx in range(batch_size):
    row = padded_tensor[row_idx]
    indices = row[row != -1]
    sparse_tensor = torch.sparse_coo_tensor(indices.unsqueeze(0), torch.ones(indices.shape), shape)
    dense = sparse_tensor.to_dense()
    rows.append(dense)
  unpadded = torch.stack(rows)
  return unpadded

In the paper they have two matrices,l et $n$ be the number of unique songs, $m$ the number of playlists and $k$ the number of unique artists:

- $P \in \mathbb{R}^{m \times n}$ where $p_i = 1$ if song $i$ is in the playlist, $p_i=0$ otherwise
- $A \in \mathbb{R}^{m \times k}$ where $a_i=1$ if the artist is present in the playlist, $a_i = 0$ otherwise

In [16]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter

CACHE = os.path.join(GDRIVE_HOME_DIR, "cache")
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, f'file://{CACHE}')

pytorch_songs_df = convert_sparse_to_indices(songs_embeddings.select("tracks", "pid"), column_name="tracks")
# songs_converter = make_spark_converter(pytorch_songs_df)
pytorch_artists_df = convert_sparse_to_indices(artists_embeddings.withColumnRenamed("tracks", "artists").select("pid", "artists"), column_name="artists")
# artist_converter = make_spark_converter(pytorch_artists_df)
songs_artists_df = pytorch_songs_df.join(pytorch_artists_df, on="pid")
pytorch_merged_dataloader = make_spark_converter(songs_artists_df)


print(pytorch_songs_df.count(), pytorch_artists_df.count(), songs_artists_df.count()) #Everything good here, this is nice!

Max number of songs: 250


Converting floating-point columns to float32


Max number of songs: 250


23/06/21 03:38:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
The median size 3296798 B (< 50 MB) of the parquet files is too small. Total size: 26620687 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///Users/dov/Desktop/big-data-project/data/cache/20230621033852-appid-local-1687311527174-72871930-8524-449d-8057-49f7bca8c717/part-00004-79d5a56f-5428-4509-a164-13f696b83de3-c000.parquet, ...


98465 98465 98465


In [17]:
# songs_embeddings.show(), artists_embeddings.show()
# artists_embeddings = artists_embeddings.withColumnRenamed("tracks", "artists").select("pid", "artists")
# df = songs_embeddings.join(artists_embeddings, on="pid").show()

Creating the dataloader for the evaluation set

# PyTorch Model

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle

class DAE_tied(nn.Module):
    def __init__(self, conf):
        super(DAE_tied, self).__init__()
        self.save_dir = conf["save"]
        if LOCAL:
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.initval_dir = conf["initval"]

        self.n_batch = conf["batch"]
        self.n_input = conf["n_input"]
        self.n_hidden = conf["hidden"]
        self.reg_lambda = conf["reg_lambda"]

        self.keep_prob = torch.tensor(conf["keep_prob"], dtype=torch.float32)
        self.input_keep_prob = torch.tensor(conf["input_keep_prob"], dtype=torch.float32)

        self.weights = {}
        self.biases = {}
        self.d_params = []

        self.z = None

    def init_weight(self):
        if self.initval_dir == 'NULL':
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden).to(self.device))
            nn.init.xavier_uniform_(self.weights['encoder_h'])
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden).to(self.device))
            nn.init.zeros_(self.biases['encoder_b'])
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input).to(self.device))
            nn.init.zeros_(self.biases['decoder_b'])
        else:
            with open(self.initval_dir, 'rb') as f:
                emb = pickle.load(f)
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(emb[0]).to(self.device))
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(emb[2]).to(self.device))
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(emb[3]).to(self.device))
        self.d_params = [self.weights['encoder_h'], self.weights['encoder_h'], self.biases['encoder_b'], self.biases['decoder_b']]


    # Building the encoder
    def encoder(self, x):
        # Encoder Hidden layer with sigmoid activation #1
        layer = torch.add(torch.matmul(x, self.weights['encoder_h']), self.biases['encoder_b'])
        layer = torch.sigmoid(layer)
        layer = torch.nn.functional.dropout(layer, p=1 - self.keep_prob)

        return layer

    # Building the decoder
    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['encoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
        # encoder_h_l2 = (torch.sum(self.weights['encoder_h']) ** 2)/2
        # decoder_b_l2 = (torch.sum(self.biases['decoder_b']) ** 2)/2
        # encoder_b_l2 = (torch.sum(self.biases['encoder_b']) ** 2)/2

        encoder_h_l1 = torch.sum(torch.abs(self.weights['encoder_h']))
        decoder_b_l1 = torch.sum(torch.abs(self.biases['decoder_b']))
        encoder_b_l1 = torch.sum(torch.abs(self.biases['encoder_b']))

        # l2 = encoder_h_l2 + decoder_b_l2 + encoder_b_l2
        l1 = encoder_h_l1 + decoder_b_l1 + encoder_b_l1
        return l1

    def forward(self, x, y):
        self.x = x.t()
        self.y = y.t()

        x_dropout = torch.nn.functional.dropout(self.x, p= 1 - self.input_keep_prob)
        reduce_sum = torch.sum(x_dropout, dim=1, keepdim=True)
        self.x_dropout = torch.div(x_dropout, reduce_sum + 1e-10)

        encoder_op = self.encoder(self.x_dropout)

        self.z = encoder_op
        self.y_pred = self.decoder(encoder_op)

        l2 = self.l2_loss()

        L = -torch.sum(self.y * torch.log(self.y_pred + 1e-10) +
                       0.55 * (1 - self.y) * torch.log(1 - self.y_pred + 1e-10), dim=1)
        self.cost = torch.mean(L) + self.reg_lambda * l2

    def save_model(self):
        params = [param.detach().numpy() for param in self.d_params]
        with open(self.save_dir, 'wb') as f:
            pickle.dump(params, f)


class DAE(DAE_tied):
    def __init__(self, conf):
        super(DAE, self).__init__(conf)

    def init_weight(self):
        if self.initval_dir == 'NULL':
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden).to(self.device))
            nn.init.xavier_uniform_(self.weights['encoder_h'])
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden).to(self.device))
            nn.init.xavier_uniform_(self.weights['decoder_h'])
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden).to(self.device))
            nn.init.zeros_(self.biases['encoder_b'])
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input).to(self.device))
            nn.init.zeros_(self.biases['decoder_b'])
        else:
            with open(self.initval_dir, 'rb') as f:
                emb = pickle.load(f)
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(emb[0]).to(self.device))
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(emb[1]).to(self.device))
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(emb[2]).to(self.device))
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(emb[3]).to(self.device))

        self.d_params = [self.weights['encoder_h'], self.weights['decoder_h'],
                         self.biases['encoder_b'], self.biases['decoder_b']]

    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['decoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
    #   encoder_h_l2 = (torch.sum(self.weights['encoder_h']) ** 2)/2
    #   decoder_b_l2 = (torch.sum(self.biases['decoder_b']) ** 2)/2
    #   encoder_b_l2 = (torch.sum(self.biases['encoder_b']) ** 2)/2
    #   decoder_h_l2 = (torch.sum(self.weights['decoder_h']) ** 2)/2

      encoder_h_l1 = torch.sum(torch.abs(self.weights['encoder_h']))
      decoder_b_l1 = torch.sum(torch.abs(self.biases['decoder_b']))
      encoder_b_l1 = torch.sum(torch.abs(self.biases['encoder_b']))
      decoder_h_l1 = torch.sum(torch.abs(self.weights['decoder_h']))


      l1 = encoder_h_l1 + decoder_b_l1 + encoder_b_l1 + decoder_h_l1

    #   l2 = encoder_h_l2 + decoder_b_l2 + encoder_b_l2 + decoder_h_l2

      return l1

In [19]:
from typing import List
def k_prec(input: torch.Tensor, eval_preds: torch.Tensor, ground_truth: torch.Tensor) -> List[float]:
    batch_size = 32
    precs = []

    for i in range(batch_size):
        input_idx = torch.nonzero(input[i] == 1).squeeze().flatten()
        num_input_songs = input_idx.shape[0]

        ground_truth_idx = torch.nonzero(ground_truth[i] == 1).squeeze().flatten()
        num_ground_truth_songs = ground_truth_idx.shape[0]

        k = num_input_songs + num_ground_truth_songs

        print(eval_preds.shape)
        top_k_preds = eval_preds[i].topk(k, dim=0)
        top_k_preds_idx = top_k_preds.indices.flatten().cpu()

        confidences = top_k_preds.values.flatten()

        already_in_playlist = np.intersect1d(top_k_preds_idx.detach().numpy(), input_idx.cpu().detach().numpy())


        top_k_preds_idx = np.array(sorted([item for item in top_k_preds_idx if item not in already_in_playlist]))[:num_ground_truth_songs]

        common_elements = np.intersect1d(top_k_preds_idx, ground_truth_idx.cpu().detach().numpy())
        num_common_elements = len(common_elements)
        precs.append(num_common_elements/num_ground_truth_songs)

    return precs

def ndcg(eval_preds: torch.Tensor, ground_truth: torch.Tensor) -> float:
    return 0

def evaluate(input: torch.Tensor, eval_preds: torch.Tensor, ground_truth: torch.Tensor) -> List[float]:
    return k_prec(input, eval_preds, ground_truth)

Creating the petastorm converters for validation set

In [20]:
pytorch_songs_eval_train_df = convert_sparse_to_indices(songs_embeddings_eval_train.withColumnRenamed("tracks", "train_tracks").select("train_tracks", "pid"), column_name="train_tracks")
pytorch_artists_eval_train_df = convert_sparse_to_indices(artists_embeddings_eval_train.withColumnRenamed("tracks", "train_artists").select("pid", "train_artists"), column_name="train_artists")
songs_artists_eval_train_df = pytorch_songs_eval_train_df.join(pytorch_artists_eval_train_df, on="pid")

pytorch_songs_eval_test_df = convert_sparse_to_indices(songs_embeddings_eval_test.withColumnRenamed("tracks", "test_tracks").select("test_tracks", "pid"), column_name="test_tracks")
pytorch_artists_eval_test_df = convert_sparse_to_indices(artists_embeddings_eval_test.withColumnRenamed("tracks", "test_artists").select("pid", "test_artists"), column_name="test_artists")
songs_artists_eval_test_df = pytorch_songs_eval_test_df.join(pytorch_artists_eval_test_df, on="pid")

eval_merged_df = songs_artists_eval_train_df.join(songs_artists_eval_test_df, on="pid")

# counter = F.udf(lambda x: len([item for item in x if item != -1]), returnType=IntegerType())
# eval_merged_df = eval_merged_df\
#     .withColumn("train_tracks_count", counter(F.col("train_tracks_indices")))\
#     .withColumn("test_tracks_count", counter(F.col("test_tracks_indices")))
# eval_merged_df = eval_merged_df.filter("train_tracks_count > 100")
eval_merged_dataloader = make_spark_converter(eval_merged_df)

Max number of songs: 250
Max number of songs: 250
Max number of songs: 250
Max number of songs: 250


Converting floating-point columns to float32


In [21]:
def validate(model: DAE_tied) -> Tuple[torch.Tensor, float, float]:
    """
    Given the model, performs an evaluation on the validation set.
    """
    ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
    SONG_SHAPE = (SONGS_VECTOR_LENGTH, )
    precs = []
    tot_k = 0
    model.eval()
    with eval_merged_dataloader.make_torch_dataloader(batch_size=conf["batch"], num_epochs = 1) as eval_dataloader:
        for batch_idx, row in enumerate(eval_dataloader):
            with torch.no_grad():
                if batch_idx == 1:
                    break #TODO: faster but less generalized, remove for the final training
                padded_eval_song_tensor = row["train_tracks_indices"]
                padded_eval_artist_tensor = row["train_artists_indices"]

                song_dense = padded_tensors_to_dense_matrix(padded_eval_song_tensor, SONG_SHAPE)
                artist_dense = padded_tensors_to_dense_matrix(padded_eval_artist_tensor, ARTIST_SHAPE)

                song_dense = song_dense.to(device)
                artist_dense = artist_dense.to(device)

                del padded_eval_song_tensor
                del padded_eval_artist_tensor

                x = torch.concat((song_dense, artist_dense), dim=1).t()
                y = torch.concat((song_dense, artist_dense), dim=1).t()

                model(x, y)

                eval_preds = model.y_pred[:, :SONGS_VECTOR_LENGTH]

                padded_eval_song_tensor_test = row["test_tracks_indices"]

                ground_truth = padded_tensors_to_dense_matrix(padded_eval_song_tensor_test, SONG_SHAPE)

                ground_truth = ground_truth.to(device)

                prec_list = evaluate(song_dense, eval_preds, ground_truth)
                precs.extend(prec_list)

        mean_prec: float = sum(precs) / len(precs)
        model.train()
        return model.cost, mean_prec


Define the validation function that is invoked during the training in order to save the model parameters that optimize the performance evaluation on the validation set.

In [22]:
def perform_validation_step(model: DAE_tied, max_prec: float, save_path:str, save: bool):
    eval_loss, prec = validate(model)

    if prec > max_prec:
        max_prec = prec
        best_params = [param.cpu().detach().numpy() for param in model.d_params]
        if save:
          with open(save_path, "wb") as f:
              pickle.dump(best_params, f)
          print(f"Best prec achieved: {prec}, parameters saved!")
        else:
          print(f"Best prec achieved: {prec}!")
    return max_prec

In [23]:
if LOCAL:
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
BEST_PARAMS_PATH = os.path.join(SAVED_MODELS, "best_params.pickle")
BEST_PARAMS_PATH_2 = os.path.join(SAVED_MODELS, "best_params_reg.pickle")

#Hyperparameters used in the paper
conf = {
    'batch': 32,
    'n_input': SONGS_VECTOR_LENGTH + ARTIST_VECTOR_LENGTH,
    'hidden': 50,
    'lr': 0.0005,
    'reg_lambda': 0.0000,
    'initval': "NULL",
    "keep_prob": 0.8,
    "input_keep_prob": 0.8, # This isn't used for now because of the .uniform()
    'save': os.path.join(SAVED_MODELS, "dae_model.pickle")
}
pretrain_model = DAE_tied(conf)
pretrain_model.init_weight()
pretrain_model.train()

DAE_tied()

In [25]:
pretrain_optimizer = optim.Adam(pretrain_model.d_params, lr=conf["lr"])

In [26]:
pretrain_model.weights['encoder_h'].device

device(type='mps', index=0)

In [27]:
torch.cuda.empty_cache()

Testing if the ordering is correct

In [28]:
# from tqdm.notebook import tqdm
# import random
# # os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
# NUM_EPOCHS = 5
# with pytorch_merged_dataloader.make_torch_dataloader(batch_size=conf["batch"], num_epochs = NUM_EPOCHS) as train_dataloader:
#     ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
#     SONG_SHAPE = (SONGS_VECTOR_LENGTH, )
#     for batch_idx, row in tqdm(enumerate(train_dataloader), desc=f"Training model", total= (NUM_PLAYLISTS / conf["batch"]) * NUM_EPOCHS):
#       # Pick random input_keep_prob between 0.5 and 0.8
#     #   pretrain_model.input_keep_prob = random.uniform(0.5, 0.8) #TODO: make this dynamic inside the conf
#       pretrain_model.input_keep_prob = random.uniform(0.5, 0.8) #TODO: make this dynamic inside the conf

#       padded_song_tensor = row["tracks_indices"]
#       padded_artist_tensor = row["artists_indices"]

#       song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)[:, 1:]
#       artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)[:, 1:]

#       song_indices = sorted(list(set(torch.nonzero(song_dense[0] == 1).squeeze().flatten().cpu().detach().tolist())))
#       print(torch.nonzero(song_dense[0] == 1))
#       songs_artists_df.show()
#       artist_indices = torch.nonzero(artist_dense == 1).squeeze().flatten()

#     #   print(song_indices)
#     #   print(artist_indices)
#       break


## Train the model

In [29]:
min_loss = 2000
max_prec = 0
best_params = []
losses = []

Pretrain with `DAE_tied`

In [30]:
from tqdm.notebook import tqdm
import random
# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
NUM_EPOCHS = 2
with pytorch_merged_dataloader.make_torch_dataloader(batch_size=1, num_epochs = NUM_EPOCHS) as train_dataloader:
    ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
    SONG_SHAPE = (SONGS_VECTOR_LENGTH, )
    for batch_idx, row in tqdm(enumerate(train_dataloader), desc=f"Training model", total= (NUM_PLAYLISTS / conf["batch"]) * NUM_EPOCHS):
      # Pick random input_keep_prob between 0.5 and 0.8
      pretrain_model.input_keep_prob = random.uniform(0.5, 0.8) #TODO: make this dynamic inside the conf

      padded_song_tensor = row["tracks_indices"]
      padded_artist_tensor = row["artists_indices"]

      song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)
      artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)

      song_dense = song_dense.to(device)
      artist_dense = artist_dense.to(device)

      rand_int = np.random.randint(2)
      if rand_int == 0:
        #Zero-out the artists
        pretrain_optimizer.zero_grad()
        # x = torch.concat((song_dense, torch.zeros_like(artist_dense)), dim=1).t()
        x = torch.concat((song_dense, artist_dense), dim=1).t()
        y = torch.concat((song_dense, artist_dense), dim=1).t()
        pretrain_model(x, y)
        loss = pretrain_model.cost
        pretrain_model.cost.backward()
        pretrain_optimizer.step()
      if rand_int == 1:
        #Zero-out the tracks
        pretrain_optimizer.zero_grad()
        # x = torch.concat((torch.zeros_like(song_dense), artist_dense), dim=1).t()
        x = torch.concat((song_dense, artist_dense), dim=1).t()
        y = torch.concat((song_dense, artist_dense), dim=1).t()
        pretrain_model(x, y)
        loss = pretrain_model.cost
        pretrain_model.cost.backward()
        pretrain_optimizer.step()
      # if rand_int == 3:
      #   #Do not zero-out anything
      #   pretrain_optimizer.zero_grad()
      #   x = torch.concat((song_dense, artist_dense), dim=1).t()
      #   y = torch.concat((song_dense, artist_dense), dim=1).t()
      #   pretrain_model(x, y)
      #   loss = pretrain_model.cost
      #   pretrain_model.cost.backward()
      #   pretrain_optimizer.step()

      if batch_idx % 20 == 0:
        losses.append(loss)

      max_prec = perform_validation_step(pretrain_model, max_prec, BEST_PARAMS_PATH, save=False)

      if batch_idx % 30 == 0:
        print(f"Loss: {loss}")
        print(f"Current max precision: {max_prec}")
        # print(f"Current confidences: ", confidences[0])

Training model:   0%|          | 0/6250.0 [00:00<?, ?it/s]

torch.Size([1, 681805])
torch.Size([1, 52469])


In [31]:
params = [param.cpu().detach().numpy() for param in pretrain_model.d_params]
with open(BEST_PARAMS_PATH_2, 'wb') as f:
  pickle.dump(params, f)

Train with `DAE` loading the pretrained `DAE_tied` model

In [35]:
conf = {
    'batch': 32,
    'n_input': SONGS_VECTOR_LENGTH + ARTIST_VECTOR_LENGTH,
    'hidden': 50,
    'lr': 0.0001,
    'reg_lambda': 0.0000,
    'initval': BEST_PARAMS_PATH,
    "keep_prob": 0.8,
    "input_keep_prob": 0.8, # This isn't used for now because of the .uniform()
    'save': os.path.join(SAVED_MODELS, "dae_model.pickle")
}
dae_model = DAE(conf)
dae_model.init_weight()
optimizer = optim.Adam(dae_model.d_params, lr=conf['lr'])

min_loss = 600
losses = []
best_params = []
max_prec = 0
FINE_TUNED_BEST_PARAMS_PATH = os.path.join(SAVED_MODELS, "final_best_params.pickle")

In [36]:
torch.cuda.empty_cache()

In [37]:
from tqdm.notebook import tqdm
import random
# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
NUM_EPOCHS = 2
with pytorch_merged_dataloader.make_torch_dataloader(batch_size=conf["batch"], num_epochs = NUM_EPOCHS) as train_dataloader:
    ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
    SONG_SHAPE = (SONGS_VECTOR_LENGTH, )
    for batch_idx, row in tqdm(enumerate(train_dataloader), desc=f"Training model", total= (NUM_PLAYLISTS / conf["batch"]) * NUM_EPOCHS):
      # Pick random input_keep_prob between 0.5 and 0.8
    #   dae_model.input_keep_prob = random.uniform(0.5, 0.8) #TODO: make this dynamic inside the conf

      dae_model.input_keep_prob = random.uniform(0.3, 1) #TODO: make this dynamic inside the conf
      dae_model.keep_prob = random.uniform(0.3, 1) #TODO: make this dynamic inside the conf

      padded_song_tensor = row["tracks_indices"]
      padded_artist_tensor = row["artists_indices"]

      song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)
      artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)

      song_dense = song_dense.to(device)
      artist_dense = artist_dense.to(device)

      rand_int = np.random.randint(2)
      if rand_int == 0:
        #Zero-out the artists
        optimizer.zero_grad()
        x = torch.concat((song_dense, artist_dense), dim=1).t()
        y = torch.concat((song_dense, artist_dense), dim=1).t()
        dae_model(x, y)
        loss = dae_model.cost
        dae_model.cost.backward()
        optimizer.step()
      if rand_int == 1:
        #Zero-out the tracks
        optimizer.zero_grad()
        x = torch.concat((song_dense, artist_dense), dim=1).t()
        y = torch.concat((song_dense, artist_dense), dim=1).t()
        dae_model(x, y)
        loss = dae_model.cost
        dae_model.cost.backward()
        optimizer.step()

      if batch_idx % 20 == 0:
        losses.append(loss)

      confidences, max_prec = perform_validation_step(dae_model, max_prec, FINE_TUNED_BEST_PARAMS_PATH, save=False)

      if batch_idx % 30 == 0:
        print(f"Loss: {loss}")
        print(f"Current max precision: {max_prec}")
        # k = torch.sum(y == 1, dim=0)
        # counts = evaluate(pretrain_model.y_pred, y.t(), k)
        # print("Precisions: ", torch.tensor(counts)/k.cpu())

Training model:   0%|          | 0/6250.0 [00:00<?, ?it/s]

torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])
torch.Size([32, 681805])


ZeroDivisionError: division by zero

Let's see how the loss decreases

In [ ]:
SAVE_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"model_new.pickle")
params = [param.cpu().detach().numpy() for param in dae_model.d_params]
with open(SAVE_MODEL_PATH, 'wb') as f:
  pickle.dump(params, f)

In [ ]:
import matplotlib.pyplot as plt
x = np.arange(len(losses))
plt.plot(x, [loss.item() for loss in losses])
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Progression')
plt.show()

# Inference

In [ ]:
from pyspark.ml.linalg import VectorUDT
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

In [ ]:
SONGS_EMBEDDINGS_PATH_TEST_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-test-train-{NUM_PLAYLISTS}.json")
SONGS_EMBEDDINGS_PATH_TEST_TEST = os.path.join(SAVED_DFS_PATH, f"nn_songs_embeddings-test-test-{NUM_PLAYLISTS}.json")

ARTISTS_EMBEDDINGS_PATH_TEST_TRAIN = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-test-train-{NUM_PLAYLISTS}.json")
ARTISTS_EMBEDDINGS_PATH_TEST_TEST = os.path.join(SAVED_DFS_PATH, f"nn_artists_embeddings-test-test-{NUM_PLAYLISTS}.json")

songs_embeddings_test_train = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH_TEST_TRAIN)
songs_embeddings_test_test = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH_TEST_TEST)

artists_embeddings_test_train = spark.read.schema(playlist_schema_mapped).json(ARTISTS_EMBEDDINGS_PATH_TEST_TRAIN)
artists_embeddings_test_test = spark.read.schema(playlist_schema_mapped).json(ARTISTS_EMBEDDINGS_PATH_TEST_TEST)

TEST_TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"nn_test_df-train-{NUM_PLAYLISTS}.json")
TEST_TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"nn_test_df-test-{NUM_PLAYLISTS}.json")

test_train_df = spark.read.schema(playlist_schema).json(TEST_TRAIN_DF_PATH)
test_test_df = spark.read.schema(playlist_schema).json(TEST_TEST_DF_PATH)

In [ ]:
def construct_prediction_df(prediction: torch.Tensor, mapping: DataFrame, top_n: int = 50) -> DataFrame:
  pred_np = prediction.detach().numpy()
  indexes = np.arange(pred_np.shape[0]) # To compensate the index start at 1
  schema = StructType([
      StructField("pos", IntegerType()),
      StructField("confidence", FloatType())
  ])
  prediction_df = spark.createDataFrame([(pos, conf) for pos, conf in zip(indexes.tolist(), pred_np.tolist())],schema)
  prediction_info = prediction_df.join(mapping, "pos")
  return prediction_info

# prediction_df = construct_prediction_df(prediction, songs_df_test)
# prediction_df.show()

In [ ]:
def remove_existing_tracks(playlist_tracks: DataFrame, recommendations_df: DataFrame) -> DataFrame:
  playlist_tracks = playlist_tracks.select("track_uri").cache()
  playlist_tracks_compatible = playlist_tracks.join(F.broadcast(recommendations_df), on="track_uri")
  playlist_tracks.unpersist()
  return recommendations_df.exceptAll(F.broadcast(playlist_tracks_compatible))

# Evaluation

In [ ]:
def precision_at_k(recommendations, ground_truth, num_of_recommendations) -> float:
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").cache()
    reccomended_relevant_tracks_count = recommended_relevant_tracks.count() #this can be top_n_results.join in order to be more performant
    recommended_relevant_tracks.unpersist()
    precision = reccomended_relevant_tracks_count / float(num_of_recommendations)

    return precision


import math
def normalized_discounted_cumulative_gain(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
  recommendations_list = recommendations.collect()
  cumulative_gain = 0

  intersection = recommendations.join(ground_truth, "track_uri").count()
  if intersection == 0: return 0

  ideal_cumulative_gain = 1 + np.array([(1 / math.log(i, 2)) for i in range(2, 2+intersection)]).sum() #TODO: replace this with sum([])
  for index, row in enumerate(recommendations_list):
    i = index + 1
    is_rel = ground_truth.filter(F.col("track_uri").isin(row.track_uri)).count() > 0
    rel = 1 if is_rel else 0
    if i == 1:
      cumulative_gain += rel
    else:
      cumulative_gain += (rel / math.log(i, 2))
  return cumulative_gain / ideal_cumulative_gain

Creating the dataloaders for the test set

In [ ]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, f'file://{CACHE}')

pytorch_songs_df_test = convert_sparse_to_indices(songs_embeddings_test_train.select("tracks"))
songs_converter_test = make_spark_converter(pytorch_songs_df_test)

pytorch_artists_df_test = convert_sparse_to_indices(artists_embeddings_test_train.select("tracks"))
artist_converter_test = make_spark_converter(pytorch_artists_df_test)

In [ ]:
#Hyperparameters used in the paper
conf = {
    'batch': 32,
    'n_input': SONGS_VECTOR_LENGTH + ARTIST_VECTOR_LENGTH,
    'hidden': 25,
    'lr': 0.0001,
    'reg_lambda': 0.0000,
    'initval': "NULL",
    "keep_prob": 0.5,
    "input_keep_prob": 0.5, # This isn't used for now because of the .uniform()
    'save': os.path.join(SAVED_MODELS, "dae_model.pickle")
}
dae_model_test = DAE(conf)
dae_model_test.init_weight()
dae_model_test.eval()

In [ ]:
ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
SONG_SHAPE = (SONGS_VECTOR_LENGTH, )
with songs_converter_test.make_torch_dataloader(batch_size=conf["batch"], num_epochs =1) as songs_dataloader:
  with artist_converter_test.make_torch_dataloader(batch_size=conf["batch"], num_epochs=1) as artists_dataloader:
    zipped_dataloaders = zip(songs_dataloader, artists_dataloader)
    for batch_idx, (song, artist) in tqdm(enumerate(zipped_dataloaders), desc=f"Evaluation...", total= (NUM_PLAYLISTS / conf["batch"])):
      padded_song_tensor = song["embedding_indices"]
      padded_artist_tensor = artist["embedding_indices"]

      song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)
      artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)

      song_dense = song_dense.to(device)
      artist_dense = artist_dense.to(device)

      x = torch.concat((song_dense, artist_dense), dim=1).t()
      y = torch.concat((song_dense, artist_dense), dim=1).t()

      dae_model_test(x,y)

      result = dae_model_test.y_pred
      break

In [ ]:
result

In [ ]:
# torch.concat((song_dense[:, 1:], artist_dense[:, 1:]), dim=1).t().shape
# (song_dense[:, 0] == 0.).all(), (artist_dense[:, 0] == 0.).all()

In [ ]:
# result = result.to("cpu")
# prediction_df = construct_prediction_df(result[10][:SONGS_VECTOR_LENGTH_TEST], songs_df_test, 20)

In [ ]:
# prediction_df.orderBy("pos").show(truncate=False)
# songs_df_test.show()

In [ ]:
# def evaluate_batch(batch_result: torch.Tensor, batch_n: int) -> Tuple[float, float]:
#   """
#   Returns the precision and NDCG for a given batch.
#   """

#   return 0,0

# def perform_evaluation(songs_dataloader, artists_dataloader, test_set):
#   """
#   Returns the precision and NDCG, averaged from all the samples in the test set
#   """
#   with songs_converter_test.make_torch_dataloader(num_epochs =1) as songs_dataloader:
#     with artist_converter_test.make_torch_dataloader(num_epochs=1) as artists_dataloader:
#       zipped_dataloaders = zip(songs_dataloader, artists_dataloader)
#       for batch_idx, (song, artist) in tqdm(enumerate(zipped_dataloaders), desc=f"Evaluation...", total= (NUM_PLAYLISTS / 32) * NUM_EPOCHS):
#         padded_song_tensor = song["embedding_indices"]
#         padded_artist_tensor = artist["embedding_indices"]

#         song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)
#         artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)

#         x = torch.concat((song_dense, artist_dense), dim=1).t()
#         y = torch.concat((song_dense, artist_dense), dim=1).t()
#         dae_model(x,y)
#         batch_result = dae_model.y_pred
#         break


In [ ]:
#Testing the first batch

results = []
# for i in tqdm(range(32)):
PID = 71860
ground_truth = test_test_df.filter(F.col("pid") == PID).select(F.explode("tracks")).select("col.*")
playlist_train_songs = test_train_df.filter(F.col("pid") == PID).select(F.explode("tracks")).select("col.*")

#Removing rare songs (that the model didn't consider)
#This may be not the best approach since the train songs or ground truth may become 0
clean_ground_truth = ground_truth.join(song_mapping, on="track_uri").cache()
clean_playlist_train_songs = playlist_train_songs.join(song_mapping, on="track_uri").cache()

# n_recommendations = ground_truth.count() or 1
n_recommendations = 500
result = result.cpu()

#The result[i] has to be aligned with the PID. i != PID.
prediction_df = construct_prediction_df(result[1][:SONGS_VECTOR_LENGTH], song_mapping, n_recommendations).cache()
clean_prediction_df = remove_existing_tracks(clean_playlist_train_songs, prediction_df)

clean_prediction_df = prediction_df.orderBy(F.col("confidence").desc()).limit(n_recommendations).cache()

prec = precision_at_k(clean_prediction_df, clean_ground_truth, n_recommendations)
gain = normalized_discounted_cumulative_gain(clean_prediction_df, clean_ground_truth, n_recommendations)
print(PID, prec, gain)
results.append((prec, gain))

ground_truth.unpersist()
clean_ground_truth.unpersist()
playlist_train_songs.unpersist()
clean_playlist_train_songs.unpersist()
prediction_df.unpersist()
clean_prediction_df.unpersist()

In [ ]:
print("Playlist train songs")
playlist_train_songs.show(truncate=False)
print("Clean playlist train songs")
clean_playlist_train_songs.show(truncate=False)
print("Ground truth songs")
ground_truth.show(truncate=False)
print("Clean ground truth songs")
clean_ground_truth.show(truncate=False)
print(f"Prediction df songs (num recommendations: {n_recommendations})")
prediction_df.show(truncate=False)
print(f"Clean Prediction df songs (num recommendations: {n_recommendations})")
clean_prediction_df.show(truncate=False)
prec, gain

In [ ]:
def average_results(results):
  prec_avg = sum(prec for prec, _ in results) / len(results)
  gain_avg = sum(gain for _, gain in results) / len(results)
  return prec_avg, gain_avg

average_results(results)